In [1]:
import torch
#for defining convolutional layer, pooling layers, fully connected layer,etc
import torch.nn as nn 
#for implementing optimizing algorithm like adam
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
#for image preprocessing and agumentation
from torchvision.transforms import v2
import torchvision.models as models
import torch.fft as fft
import matplotlib.pyplot as plt

In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
class pneumoniaDataset(Dataset):
    def __init__(self, image_path, device):
        self.image_path = image_path
        self.device = device
        self.data = []
        self.transform = v2.Compose([
            v2.ToImage(),
            v2.ToDtype(torch.float32, scale=True),
            v2.Resize((227,227)),
            v2.Normalize(mean=[0.5,], std=[0.5,]),
            v2.Grayscale(num_output_channels=3)
        ])
        self.load_data()

    def load_data(self):
        for idx, label in enumerate(os.listdir(os.path.join(self.image_path))):
            print(os.path.join(self.image_path, label))
            for img_file in tqdm(os.listdir(os.path.join(self.image_path, label))):
                img = cv2.imread(os.path.join(self.image_path, label, img_file), cv2.IMREAD_GRAYSCALE)

                img = self.transform(img)

                idx = torch.tensor([idx])
                self.data.append((img.to(self.device), idx.to(self.device)))
    
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        img,label= self.data[idx]
        label=label.squeeze()
        return img,label

In [5]:
# since i was struglling with automatically appearing of .DS_store in my folder
# which lead to unwanted classifying as class and  labeling it as 0 
# Remove .DS_Store files from the folder and subfolders
for root, dirs, files in os.walk('data'):
    if '.DS_Store' in files:
        os.remove(os.path.join(root, '.DS_Store'))

In [6]:
# Initialize the dataset
train_path=os.path.join('data/train_set')
# validation_path=os.path.join('data/val_set')
# test_path=os.path.join('data/test_set')
train_dataset = pneumoniaDataset(image_path=train_path, device=device)
# validation_dataset = pneumoniaDataset(image_path=validation_path, device=device)
# test_dataset = pneumoniaDataset(image_path=test_path, device=device)

data/train_set/bacterial


100%|██████████████████████████████████████| 2700/2700 [00:10<00:00, 257.48it/s]


data/train_set/normal


100%|██████████████████████████████████████| 2943/2943 [00:24<00:00, 118.61it/s]


data/train_set/viral


100%|██████████████████████████████████████| 1491/1491 [00:07<00:00, 196.64it/s]


In [7]:
# len(train_dataset), len(validation_dataset)

In [8]:
#dataloaders
batch_size = 24
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
# validation_dataloader=DataLoader(validation_dataset,batch_size=batch_size,shuffle=False)
# test_dataloader=DataLoader(test_dataset,batch_size=16,shuffle=False)

In [9]:
# print(len(validation_dataset))
# image, label = train_dataset[6000] 
# print(image.shape)         
# print(label)  
# print(label.shape) 

In [10]:
# # Define the model
# class AlexNet(nn.Module):
#     def __init__(self,input_channels,number_of_classes):
#         super(AlexNet, self).__init__() 
#         # Convolutional layers
#         self.conv1 = nn.Conv2d(input_channels, 96, kernel_size=11, stride=4, padding=2)
#         self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
#         self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
#         self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
#         self.conv5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)

#         # Fully connected layers
#         self.fc1 = nn.Linear(9216, 4069)
#         self.fc2 = nn.Linear(4069, 4069)
#         self.fc3 = nn.Linear(4069, number_of_classes)

#         # Other
#         self.flatten = nn.Flatten()
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
#         self.norm = nn.LocalResponseNorm(size=5, k=2)
#         self.droput = nn.Dropout(0.5)
#         self.relu = nn.ReLU()
#         self.logsoftmax = nn.LogSoftmax(dim=1)

#     def forward(self, x):
#         x = self.maxpool(self.norm(self.relu(self.conv1(x))))  # (B, 96, 27, 27)
#         x = self.maxpool(self.norm(self.relu(self.conv2(x))))  # (B, 256, 13, 13)
#         x = self.relu(self.conv3(x))                           # (B, 384, 13, 13)
#         x = self.relu(self.conv4(x))                           # (B, 384, 13, 13)
#         x = self.maxpool(self.relu(self.conv5(x)))             # (B, 256, 6, 6)
#         x = self.flatten(x)                                    # (B, 9216)
#         x = self.droput(self.relu(self.fc1(x)))                # (B, 4096)
#         x = self.droput(self.relu(self.fc2(x)))                # (B, 4096)
#         x = self.logsoftmax(self.fc3(x))                                       # (B, num_classes)
#         return x

In [11]:
# # model=AlexNet(3,3)
# model_cont = AlexNet(3,3)

In [12]:
# model_cont.load_state_dict(torch.load('alexNetModel.pth'))

In [13]:
model = models.alexnet(weights=None)
model.classifier[6] = nn.Linear(in_features=4096, out_features=3, bias=True)

model = model.to(device)

In [14]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [15]:
# number_of_epochs=10
learning_rate = 0.0001
weight_decay = 0.000001

#loss function
criterion = nn.CrossEntropyLoss()

#optimizer
# optimizer=optim.Adam(model.parameters(),learning_rate)
optimizer = optim.Adam(model.parameters(),lr=learning_rate, weight_decay=weight_decay)

In [16]:
dummy_ip = torch.randn(1,3,227,227)
op = model(dummy_ip)
_, op1 = torch.max(op, 1)
op1.item(), op1

(2, tensor([2]))

In [19]:
def train(model, train_dl, optimizer, loss_fn, epochs):
    best_acc = 0.0
    acc_list = []
    loss_list = []
    model.train()
    for epoch in range(epochs):
        running_corrects = 0.0
        running_loss = 0.0
        total_entries = 0
        print(f'Epoch: [{epoch+1}/{epochs}]')
        for images, labels in tqdm(train_dl):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)

            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total_entries += labels.size(0)
            
        epoch_loss = running_loss / total_entries
        epoch_acc = running_corrects / total_entries
        acc_list.append(epoch_acc)
        loss_list.append(epoch_loss)
        print(f"Epoch Loss: {epoch_loss:.4f}, Epoch Accuracy: {epoch_acc:.4f}")
    print('Training Complete.')
    return acc_list, loss_list

In [20]:
acc_list, loss_list = train(model, train_dataloader, optimizer, criterion, epochs=10)

Epoch: [1/10]


100%|█████████████████████████████████████████| 298/298 [03:39<00:00,  1.36it/s]


Epoch Loss: 0.4648, Epoch Accuracy: 0.7998
Epoch: [2/10]


 11%|████▊                                     | 34/298 [00:25<03:16,  1.35it/s]


KeyboardInterrupt: 

In [ ]:
# total_step = len(train_dataloader)

# # model.to(device)
# model_cont.to(device)

# for epoch in range(number_of_epochs):
#     # model.train()
#     model_cont.train()
#     for images, labels in tqdm(train_dataloader):  
#         images = images.to(device)
#         labels = labels.to(device)
#         labels= labels.long()

#         # Forward pass
#         # outputs = model(images)
#         outputs = model_cont(images)
#         loss = criterion(outputs, labels)

#         # Backward and optimize
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     # model.eval()
#     model_cont.eval()
#     val_loss = 0.0
#     val_correct = 0
#     val_total = 0
    
#     with torch.no_grad():
#         for images,labels in validation_dataloader:
#             images = images.to(device)
#             labels = labels.to(device)
#             labels= labels.long()
            
#             # outputs = model(images)
#             outputs = model_cont(images)
#             loss = criterion(outputs,labels)
#             val_loss += loss.item()

#             _, predicted = torch.max(outputs.data, 1)
#             val_total +=labels.size(0)
#             val_correct += (predicted ==labels).sum().item()

#     # Calculate validation metrics
#     avg_val_loss = val_loss / len(validation_dataloader)
#     val_accuracy = 100 * val_correct / val_total

#     print(f'Epoch [{epoch+1}/{number_of_epochs}], '
#           f'Train Loss: {loss.item():.4f}, '
#           f'Val Loss: {avg_val_loss:.4f}, '
#           f'Val Acc: {val_accuracy:.2f}%')

# print('Finished training')

In [ ]:
torch.save(model_cont.state_dict(), './alexNetModel.pth')
# torch.save(model.state_dict(), './alexNetModelcont.pth')